## Testing of the Graph Environment

Imports

In [1]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym
import pickle
from datetime import datetime

import ray

import warnings
warnings.filterwarnings('ignore')


In [2]:
#from ray.rllib.agents.dqn import DQNTrainer, DEFAULT_CONFIG, APEX_DEFAULT_CONFIG
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

Import the Environment

In [3]:
sys.path.insert(0,"")

from ManhattanGraph import ManhattanGraph
from gym_graphenv.envs.GraphworldManhattan import GraphEnv

In [4]:
env_config = {'pickup_hub_index':2,
        'delivery_hub_index':3,
        'pickup_timestamp': datetime(2016, 6, 14, 22, 0, 0),
        'delivery_timestamp': datetime(2016, 6, 14, 1, 0, 0) 
}

with open('env_config.pkl', 'wb') as f:
    pickle.dump(env_config, f)

In [5]:
env=GraphEnv()


Run random steps without Ray Trainer

In [6]:
def run_one_episode (env):
    env.reset()
    print("reset done")
    sum_reward = 0
    for i in range(30):
        action = env.action_space.sample()
        #print(action)
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("DELIVERY DONE! sum_reward: ",sum_reward)
            break

        print("sum_reward: ",sum_reward)
        # print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

for i in range(1): 
    run_one_episode (env)

Reset with config
final hub: 3
current position: 2
reset done
action ==  29  New Position 29
2 -> 29 60.0
Step() Execution time: 0.4407789707183838 seconds
sum_reward:  -60.0
action ==  53  New Position 53
29 -> 53 60.0
Step() Execution time: 0.27228403091430664 seconds
sum_reward:  -120.0
action ==  54  New Position 54
53 -> 54 60.0
Step() Execution time: 0.26592206954956055 seconds
sum_reward:  -180.0
action ==  28  New Position 28
54 -> 28 60.0
Step() Execution time: 0.3884148597717285 seconds
sum_reward:  -240.0
action ==  34  New Position 34
28 -> 34 60.0
Step() Execution time: 0.25540876388549805 seconds
sum_reward:  -300.0
action ==  35  New Position 35
34 -> 35 60.0
Step() Execution time: 0.25503015518188477 seconds
sum_reward:  -360.0
action ==  33  New Position 33
35 -> 33 60.0
Step() Execution time: 0.29365992546081543 seconds
sum_reward:  -420.0
action ==  42  New Position 42
33 -> 42 60.0
Step() Execution time: 0.2714529037475586 seconds
sum_reward:  -480.0
action ==  60  

Initialize Ray

In [7]:
ray.init()

2022-05-11 23:52:04,354	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-05-11_23-51-59_217896_32840/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-05-11_23-51-59_217896_32840/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-05-11_23-51-59_217896_32840',
 'metrics_export_port': 54855,
 'gcs_address': '127.0.0.1:60056',
 'address': '127.0.0.1:60056',
 'node_id': '743fa239a6929809125b0e177d5f6a7a6424371c755d4aca265f6e42'}

Set DQN trainer configuration

In [11]:
dqn_config = DEFAULT_CONFIG.copy()
dqn_config['num_workers'] = 1
dqn_config["train_batch_size"] = 400
dqn_config["gamma"] = 0.95
dqn_config["n_step"] = 10
dqn_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#dqn_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

Set DQN - Rainbow configuration

In [9]:
rainbow_config = DEFAULT_CONFIG.copy()

rainbow_config = DEFAULT_CONFIG.copy()
rainbow_config['num_workers'] = 1
rainbow_config["train_batch_size"] = 400
rainbow_config["gamma"] = 0.95
rainbow_config["n_step"] = 10
rainbow_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#rainbow_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

#rainbow parameters

# N-step Q learning
rainbow_config["n_step"]= 5 #[between 1 and 10]
# Whether to use noisy network
rainbow_config["noisy"] = True
# Number of atoms for representing the distribution of return. When
# this is greater than 1, distributional Q-learning is used.
# the discrete supports are bounded by v_min and v_max
rainbow_config["num_atoms"] =2 #[more than 1]
rainbow_config["v_min"] =-100
rainbow_config["v_max"]=100 # (set v_min and v_max according to your expected range of returns).

Set DQN - Ape-X configuration 

In [10]:
#apex_config=APEX_DEFAULT_CONFIG.copy()

Initialize the DQN trainer, Rainbow DQN and DQN Ape-X

In [13]:
ppo_trainer=PPOTrainer(DEFAULT_CONFIG,GraphEnv)

(RolloutWorker pid=32879) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=32878) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=32878) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=32878) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=32878) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=32879) UserWarning: The `get_nearest

(RolloutWorker pid=32878) Reset with config
(RolloutWorker pid=32878) final hub: 3
(RolloutWorker pid=32878) current position: 2
(RolloutWorker pid=32879) Reset with config
(RolloutWorker pid=32879) final hub: 3
(RolloutWorker pid=32879) current position: 2


2022-05-11 23:57:54,296	INFO trainable.py:127 -- Trainable.setup took 31.835 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-05-11 23:57:54,346	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [ ]:
trainer_dqn = DQNTrainer(dqn_config,GraphEnv)

In [ ]:
trainer_rainbow=DQNTrainer(rainbow_config,GraphEnv )


In [ ]:
#trainer_apex=DQNTrainer(apex_config,GraphEnv )

Define the path where the results of the trainer should be saved

In [ ]:
checkpoint_root = "tmp/dqn/graphworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Run the trainer

In [15]:
results = []
episode_data = []
episode_json = []
n_iter=10

for n in range(n_iter):
    result = ppo_trainer.train()
    results.append(result)
    print("TEST",n)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = ppo_trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

StopIteration: 

In [ ]:
results

Testing of the trainer

In [ ]:
trainer.restore(file_name)
env = gym.make("graphworld-v0")
state = env.reset()

In [ ]:
sum_reward = 0
n_step = 20
for step in range(n_step):
    action = trainer_dqn.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward
    #env.render()
    if done == 1:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

In [ ]:
ray.shutdown()